In [43]:
from collections import *
from numpy import *
import numpy as np
import string
from nltk.stem.snowball import SnowballStemmer
import nltk
import song_parsing

In [44]:
N_CLUSTERS = 3

desired_pos = ['IN', 'JJ', 'JJR', 'JJS', 'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'RB', 'RBR', 'RBS', 'VB', 
    'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']

def get_all_words_and_word_counts(songs):
    stemmer = SnowballStemmer("english")
    all_words = set([])
    song_counters = {}
    for song in songs:
        words = []
        for line in songs[song].split('\n'):
            t = nltk.word_tokenize(line.decode('utf-8'))
            tagged = nltk.pos_tag(t)
            for word in tagged:
                if word[1] in desired_pos:
                    words.append(word[0])
                                
        stemmed = []
        for word in words: 
            try:
                stem = stemmer.stem(word)
            except:
                stem = word
            stemmed.append(stem)
                            
        counter = collections.Counter(stemmed)
        song_counters[song] = counter
        all_words = all_words.union( counter.iterkeys() )

    all_words = sorted(all_words)
    return all_words, song_counters
    
def get_all_representations_as_matrix(songs, song_counters, all_words):
    all_song_representations = [[] for i in range(len(songs))]
    for word in all_words:
        for song_index, song in enumerate(songs):
            all_song_representations[song_index].append(song_counters[song[0]][word]) 
    return all_song_representations
    
def get_clusters(songs):
    all_words, song_counters = get_all_words_and_word_counts(songs)
    songs = list(songs.iteritems())
    return np.array(get_all_representations_as_matrix(songs, song_counters, all_words)), all_words

In [45]:
songs = song_parsing.get_all_song_lyrics("Taylor Swift")

In [46]:
all_song_reps, all_words = get_clusters(songs)

In [48]:
import lda
all_song_reps.shape
all_song_reps.sum()
model = lda.LDA(n_topics=3, n_iter=1500, random_state=1)
model.fit(all_song_reps)  # model.fit_transform(X) is also available
topic_word = model.topic_word_  # model.components_ also works
n_top_words = 8
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(all_words)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: in was love on of so back never
Topic 1: got now in na [ la 'm shake
Topic 2: n't do know 's 'm is just of
